In [17]:
!git clone https://github.com/AI4Finance-LLC/FinRL-Library.git

Cloning into 'FinRL-Library'...
remote: Enumerating objects: 2824, done.
remote: Total 2824 (delta 0), reused 0 (delta 0), pack-reused 2824
Receiving objects: 100% (2824/2824), 30.68 MiB | 22.50 MiB/s, done.
Resolving deltas: 100% (1698/1698), done.


In [2]:
cd /content/FinRL-Library/

/content/FinRL-Library


In [3]:
pip install -r requirements.txt

     |████████████████████████████████| 153kB 12.8MB/s 
     |████████████████████████████████| 5.5MB 26.2MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=064b9360de145fb2e7559da6d09b6b2c3131ce40cfcc442555fce0aa09ef1ac6
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [7]:
!pip install pyfolio

     |████████████████████████████████| 92kB 6.5MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for pyfolio: filename=pyfolio-0.9.2-cp37-none-any.whl size=88667 sha256=fb182364e54c2abccbeca173a5fb99a8ed274b3281792353cee84e9c3b74b26e
  Stored in directory: /root/.cache/pip/wheels/0d/a8/7d/ac33fe6bf6f3db370291c9de303460d00b8f97cc40a1d80f72
  Created wheel for empyrical: filename=empyrical-0.5.5-cp37-none-any.whl size=39764 sha256=e1f53983fc78932725b4197fe6e6ec11d4ee6299ba85b1a17a45c4456d7b06a6
  Stored in directory: /root/.cache/pip/wheels/ea/b2/c8/6769d8444d2f2e608fae2641833110668d0ffd1abeb2e9f3fc
Successfully built pyfolio empyrical


In [8]:
from finrl.trade.backtest import backtest_stats

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [9]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

# import and if any module is not installed, install it using pip
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import datetime
import os
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats#, BaselineStats, BackTestPlot
import sys
sys.path.append("../FinRL-Library")

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-k201tix5
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-k201tix5
  Created wheel for finrl: filename=finrl-0.3.0-cp37-none-any.whl size=38201 sha256=ce46412a207a7d5a267dffb9e9e0505cc761c27b93b0988ff7ddad6d912e202c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9p57cif2/wheels/9c/19/bf/c644def96612df1ad42c94d5304966797eaa3221dffc5efe0b
Successfully built finrl


In [10]:
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [11]:
data_df = YahooDownloader(start_date = '2009-01-01',
                          end_date = '2021-01-01',
                          ticker_list = ['AAPL']).fetch_data()
data_df.head()

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3021, 8)


,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4
1,2009-01-05,3.327500,3.435000,3.311071,2.909563,1181608400,AAPL,0
2,2009-01-06,3.426786,3.470357,3.299643,2.861573,1289310400,AAPL,1
3,2009-01-07,3.278929,3.303571,3.223571,2.799739,753048800,AAPL,2
4,2009-01-08,3.229643,3.326786,3.215714,2.851728,673500800,AAPL,3


In [12]:
## let’s store the technical indicator column names in config.py
tech_indicator_list=config.TECHNICAL_INDICATORS_LIST
## you can add more technical indicators
## visit https://github.com/jealous/stockstats for different names
tech_indicator_list=tech_indicator_list+['kdjk','open_2_sma','boll','close_10.0_le_5_c','wr_10','dma','trix']
print(tech_indicator_list)

#Passing parameter to FeatureEngineer for adding technical indicators.
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = tech_indicator_list,
                    use_turbulence=False,
                    user_defined_feature = False)

data_df = fe.preprocess_data(data_df)
data_df.head()

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma', 'kdjk', 'open_2_sma', 'boll', 'close_10.0_le_5_c', 'wr_10', 'dma', 'trix']
Successfully added technical indicators


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,kdjk,open_2_sma,boll,close_10.0_le_5_c,wr_10,dma,trix
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,-6.299850,3.067143,2.791740,1.0,218.899551,0.0,0.670734
1,2009-01-05,3.327500,3.435000,3.311071,2.909563,1181608400,AAPL,0,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,-15.368278,3.197322,2.850651,2.0,133.505133,0.0,0.670734
2,2009-01-06,3.426786,3.470357,3.299643,2.861573,1289310400,AAPL,1,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,-24.222698,3.377143,2.854292,3.0,141.931537,0.0,0.391304
3,2009-01-07,3.278929,3.303571,3.223571,2.799739,753048800,AAPL,2,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,-34.930948,3.352857,2.840654,4.0,156.347447,0.0,0.195393
4,2009-01-08,3.229643,3.326786,3.215714,2.851728,673500800,AAPL,3,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,-38.029528,3.254286,2.842869,5.0,144.226688,0.0,0.125125


In [13]:
train = data_split(data_df, start = '2009-01-01', end = '2019-01-01')
trade = data_split(data_df, start = '2019-01-01', end = '2021-01-01')

In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock data Dimensions: {stock_dimension}, State Spaces: {state_space}")
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

Stock data Dimensions: 1, State Spaces: 11
<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [20]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=30000)

{'batch_size': 64, 'buffer_size': 500000, 'learning_rate': 0.0001}
Using cuda device
Logging to tensorboard_log/ddpg/ddpg_1
----------------------------------
| environment/        |          |
|    portfolio_value  | 7.86e+05 |
|    total_cost       | 5.3e+03  |
|    total_reward     | 6.86e+05 |
|    total_reward_pct | 686      |
|    total_trades     | 2515     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 87       |
|    time_elapsed     | 115      |
|    total timesteps  | 10064    |
| train/              |          |
|    actor_loss       | 69.6     |
|    critic_loss      | 308      |
|    learning_rate    | 0.0001   |
|    n_updates        | 7548     |
----------------------------------
----------------------------------
| environment/        |          |
|    portfolio_value  | 8.6e+05  |
|    total_cost       | 99.9     |
|    total_reward     | 7.6e+05  |
|    total_reward_pct | 760      |
|    total_trades     | 2515     |
|